## 카드 소비 데이터

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import dask.dataframe as dd
import vaex

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as spst

In [3]:
sample_data = '../data/gm_card.csv'
data = '../data/gyeonggi_card_2023.csv'

In [4]:
# Dask로 1억건 데이터 LOAD
card_df = dd.read_csv(data,encoding='utf-8')


In [5]:
card_df.head()

,ta_ymd,cty_rgn_no,admi_cty_no,card_tpbuz_cd,card_tpbuz_nm_1,card_tpbuz_nm_2,hour,sex,age,day,amt,cnt
0,20230101,41210,41210510,D05,소매/유통,선물/완구,3,M,5,7,57576,2
1,20230101,41210,41210510,D11,소매/유통,종합소매점,3,F,7,7,8523,2
2,20230101,41210,41210510,D11,소매/유통,종합소매점,7,F,7,7,8023,2
3,20230101,41210,41210510,F02,생활서비스,미용서비스,3,M,4,7,28532,2
4,20230101,41210,41210510,F02,생활서비스,미용서비스,4,F,7,7,74074,2


In [6]:
card_df.isna().sum()

Dask Series Structure:
npartitions=1
admi_cty_no    int64
ta_ymd           ...
dtype: int64
Dask Name: dataframe-sum-agg, 4 graph layers

In [10]:
card_df.count().compute(scheduler='processes', num_workers=2)

ta_ymd             157871189
cty_rgn_no         157871189
admi_cty_no        157871189
card_tpbuz_cd      157871189
card_tpbuz_nm_1    157871189
card_tpbuz_nm_2    157871189
hour               157871189
sex                157871189
age                157871189
day                157871189
amt                157871189
cnt                157871189
dtype: int64

In [11]:
#card_df.sort_values(by=['ta_ymd', 'card_tpbuz_nm_1','card_tpbuz_nm_2'])

In [ ]:
#card_df.groupby(['ta_ymd', 'card_tpbuz_nm_1','card_tpbuz_nm_2', 'sex'])

### 코드를 위한 데이터 분리 및 결합

In [12]:
code_mapping = card_df[["card_tpbuz_cd",	"card_tpbuz_nm_1",	"card_tpbuz_nm_2"	]].drop_duplicates()

In [13]:
#해당 일, 특정행정동의 업종별, 연령별, 나이별 매출액 평균과 매출 건수
grouped_df = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd", "sex", "age"]).agg({"amt": "mean", "cnt": "sum"}).reset_index()

In [16]:
grouped_df.sort_values(by=["ta_ymd", "card_tpbuz_cd","admi_cty_no"])

,ta_ymd,admi_cty_no,card_tpbuz_cd,sex,age,amt,cnt
npartitions=1,,,,,,,
,int64,int64,object,object,int64,float64,int64
,...,...,...,...,...,...,...


In [7]:
time_df = card_df.groupby(["ta_ymd", "admi_cty_no", "card_tpbuz_cd"]).agg({"amt": "mean"}).reset_index()
time_df2 = time_df.groupby(['ta_ymd',	'card_tpbuz_cd']).agg({"amt": "mean"}).reset_index()

In [8]:
time_df2.head()

,ta_ymd,card_tpbuz_cd,amt
0,20230101,D05,1.270882e+05
1,20230101,D11,1.129375e+06
2,20230101,F02,1.892528e+05
3,20230101,Q01,1.828111e+05
4,20230101,Q15,4.048536e+05


In [18]:
#grouped_df['ta_ymd'] = pd.to_datetime(grouped_df['ta_ymd'], format='%Y%m%d')
# 다음날 매출을 라벨로 추가
#grouped_df['next_day_sales'] = grouped_df.groupby(['admi_cty_no', 'card_tpbuz_cd', 'sex', 'age'])['amt'].shift(-1)
# 데이터 확인
# grouped_df[(grouped_df['ta_ymd'] == '2023-01-01') & 
#                              (grouped_df['admi_cty_no'] == 41210510) & 
#                              (grouped_df['card_tpbuz_cd'] == 'F02')]

### 날짜 타입 변환

In [15]:
# time_df2['ta_ymd'] = time_df2['ta_ymd'].astype(str)
time_df2['ta_ymd'] = pd.to_datetime(time_df2['ta_ymd'])

ValueError: Length of values (30271) does not match length of index (2)

## 모델 학습 -> AutoML 활용

In [13]:
# autogluon 사용
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [14]:
train_data = TimeSeriesDataFrame.from_data_frame(time_df2, id_column = "card_tpbuz_cd", timestamp_column="ta_ymd")

ValueError: 0'th time-series in data must be a dict, got<class 'str'>

In [23]:
train_data

NameError: name 'train_data' is not defined

In [24]:
train_data = TimeSeriesDataFrame.from_data_frame(time_df2, id_column = "card_tpbuz_cd", timestamp_column="ta_ymd")

ValueError: 0'th time-series in data must be a dict, got<class 'str'>

In [25]:
train_data

NameError: name 'train_data' is not defined

In [26]:
predictor = TimeSeriesPredictor(label="amt", prediction_length=28, eval_metric="RMSE", freq="D")

In [ ]:
predictor.fit(train_data, random_seed = 42)